In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo("kZjSAa18eJo")

## Why SQLite?
* Zero configuration
    * no need to mess with complex administration (e.g. MySQL)
* Portability
    * Windows, Linux, OS X, Solaris, HPUX, AIX, Palm. 16, 32, 64 bit architecture
* Compactness: 
    * small memory footprint
* Simplicity: 
    * good separation between compiler and storage.
* Liberal Licensing: 
    * Free. No copyright claim.
* Reliable: 
    * one line of testing for every line of code
* Convenient: 
    * lots of features to facilitate ease of use
* Integrated directly into Python (since 2.5)

## SQLite Limitations

* Will be slower than more complex databases for complex queries on large database
* No foreign key constraints
* One write at a time (no concurrency)

## Making a Connection
* The first step in interacting with a database is to connect to it via the connect command. 
* This creates a Connection object that represents the database.

In [ ]:
import sqlite3 as sqlite
import os
DATADIR = os.path.join(os.path.expanduser("~"),"6521","mdcrc6521spring2016","DATA")
print(os.path.exists(DATADIR))
conn = sqlite.connect(os.path.join(DATADIR,"seinfeldFood.sqlite"))

In [ ]:
help(conn.cursor)

### Connection Highlights
* **cursor():** Creates a cursor for executing a command
* **execute(), executescript(), executemany():** shortcuts to do the work of a cursor() without actually creating a cursor
* Use the name ":memory:" to create a database in RAM

In [ ]:
conn2 = sqlite.connect(":memory:")

### Cursors
* Cursors are used to execute SQL queries and to examine the results.

In [ ]:
curs = conn.cursor()
curs.

In [ ]:
help(curs.execute)

In [ ]:
help(curs.executemany)

In [ ]:
help(curs.executescript)

In [ ]:
help(curs.fetchall)

In [ ]:
help(curs.setoutputsize)

In [ ]:
help(curs.fetchmany)

In [ ]:
help(curs.close)

In [ ]:
help(curs.fetchone)

## Some Basic SQL Commands in SQLite
* [CREATE TABLE](https://www.sqlite.org/lang_createtable.html)
* Basic syntax: "``CREATE TABLE <TABLE_NAME> (COLUMN NAME COLUMN TYPE)``

In [ ]:
conn = sqlite.connect(":memory:")
cursor = conn.cursor()
cursor.execute("""CREATE TABLE run_args (run_date text,
                                         label text,
                                         args text)""")

* If I try to create a table that already exists, I'll get an error

In [ ]:
cursor.execute("""CREATE TABLE run_args (run_date text,
                                         label text,
                                         args text)""")

## SQL let us insert a conditional for the creation

In [ ]:
cursor.execute("""CREATE TABLE IF NOT EXISTS run_args (run_date text,
                                                       label text,
                                                       args text)""")

* I could also use an exception

### Foreign Keys

* [Relational database design](http://geekgirls.com/2011/09/databases-from-scratch-iii-relational-design-process/) relies on splitting data between different tables
* We have to be able to reference entries in one table to entries in another. We do this with **foreign keys**

#### Syntax
* ``FOREIGN KEY (COLUMN NAME) REFERENCES SOME_TABLE(SOME_COLUMN)``
* EXAMPLE: 
    * ``FOREIGN KEY (result) REFERENCES pyConTextNLP_results(rowid))``


In [ ]:
cursor.execute("""create table if not exists run_args (run_date text,
                                                            label text,
                                                            args text)""")
cursor.execute("""CREATE TABLE IF NOT EXISTS class_schema (schema text)""")

cursor.execute(
        """CREATE TABLE IF NOT EXISTS pyConTextNLP_results
            (report_number int, 
            run_args int, 
            schema int,
            target_category text,
            classification int, 
            most_positive_target text,
            FOREIGN KEY (run_args) REFERENCES run_args(rowid),
            FOREIGN KEY (schema) REFERENCES class_schema(rowid))""")

cursor.execute(
        """CREATE TABLE IF NOT EXISTS pyConTextNLP_severity
            (result int,
            phrase text,
            svalue text,
            units text,
            FOREIGN KEY (result) REFERENCES pyConTextNLP_results(rowid))""")

* **INSERT**

In [ ]:
cursor.execute("""INSERT INTO run_args(run_date,label,args) VALUES (?,?,?)""",
                                    (run_time, result_label, outString,))

* **SELECT**

In [ ]:
cursor.execute("""SELECT * from run_args""")
cursor.execute("""SELECT rowid,label FROM run_args""")
cursor.execute("""SELECT rowid FROM run_args WHERE run_date = ?""",(run_time,))

### Some Important Notes
* Any changes done  have to be committed with the connection object **commit()** method
* Never use Python string formatting to directly generate an SQL script (security risk)

In [ ]:
query = """INSERT INTO class_participants (name,role) VALUES (%s,%s)"""%("Brian","Teacher")
curs2.execute('select * from stocks where symbol=?', (symbol,))

* After an execute, the cursor object can be used as an iterator to access the results. 
* Alternatively results can be accessed via 
    * **fetchone()**: fetch next result
    * **fetchall()**: fetch all results
    * **fetchmany()**: fetch next N results



In [ ]:
conn = sqlite.connect(":memory:")
cursor = conn.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS class_participants
                (name text, role text)""")
cursor.execute("""INSERT INTO class_participants (name,role) VALUES (?,?)""",
               ("Brian","teacher"))

roster = [("Steven","TA"),("Bob","TA"),
          ("Kathy","Observer"),("Arun","Student"),
          ("Jody","Student")]
cursor.executemany("""INSERT INTO class_participants (name,role) VALUES (?,?)""",roster)

In [ ]:
cursor.execute("""INSERT INTO class_participants (name) VALUES (?)""",("Homer",))

In [ ]:
cursor.execute("""SELECT * FROM class_participants""")

for r in cursor.fetchall():
    print( r)

## Python to SQLite3 Data type Conversion

|Python type|SQLite type|
|----------:|:----------|
|None | NULL|
|int | INTEGER|
|long | INTEGER|
|float | REAL|
|str (UTF8-encoded) | TEXT|
|unicode | TEXT|
|buffer | BLOB|

# For Future Reference

We have not discussed classes yet, so this material will not be used.

## Saving User defined Types
* Strategy: 
    * Define method to adapt the object to a database type (e.g. string); 
    * Save the string to the database
    * Read the string from the database
    * Define a function to create an objects from the string
    
### [Converting Objects to String](https://docs.python.org/3.5/library/sqlite3.html#letting-your-object-adapt-itself)
* Using ``__conform__`` to have object adapt itself


In [ ]:

import sqlite3 as sqlite

class Point(object):
    def __init__(self, x, y):
        self.x, self.y = x, y
    def __str__(self):
        return "(%f;%f)" % (self.x, self.y) 
    def __repr__(self):
        return "(%f;%f)" % (self.x, self.y)
    def __conform__(self, protocol):
        if protocol is sqlite.PrepareProtocol:
            return "%f;%f" % (self.x, self.y)

con = sqlite.connect(":memory:")
cur = con.cursor()

p = Point(4.0, -3.2)
cur.execute("select ?", (p,))
print (cur.fetchone()[0])

## Registering an adapter callable

In [ ]:
import sqlite3 as sqlite

def adapt_point(point):
    return "%f,%f" % (point.x, point.y)

sqlite.register_adapter(Point, adapt_point)

con = sqlite.connect(":memory:")
cur = con.cursor()

p = Point(4.0, -3.2)
cur.execute("select ?", (p,))
s=  cur.fetchone()[0]
print( s)